In [ ]:
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
# from airflow import DAG
import requests
import pandas as pd

In [ ]:
import pandas as pd

df = pd.read_parquet("hf://datasets/AiresPucrs/tmdb-5000-movies/data/train-00000-of-00001-6db04ab1c75d6817.parquet")

In [ ]:
df.head()

,id,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,5,4000000,"[{""id"": 80, ""name"": ""Crime""}, {""id"": 35, ""name...",None,"[{""id"": 612, ""name"": ""hotel""}, {""id"": 613, ""na...",en,Four Rooms,It's Ted the Bellhop's first night on the job....,22.876230,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...",...,4300000,98.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,530,"[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,11000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 28, ""...",http://www.starwars.com/films/star-wars-episod...,"[{""id"": 803, ""name"": ""android""}, {""id"": 4270, ...",en,Star Wars,Princess Leia is captured and held hostage by ...,126.393695,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...",...,775398007,121.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,6624,"[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,94000000,"[{""id"": 16, ""name"": ""Animation""}, {""id"": 10751...",http://movies.disney.com/finding-nemo,"[{""id"": 494, ""name"": ""father son relationship""...",en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",85.688789,"[{""name"": ""Pixar Animation Studios"", ""id"": 3}]",...,940335536,100.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,6122,"[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,55000000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",None,"[{""id"": 422, ""name"": ""vietnam veteran""}, {""id""...",en,Forrest Gump,A man with a low IQ has accomplished great thi...,138.133331,"[{""name"": ""Paramount Pictures"", ""id"": 4}]",...,677945399,142.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,7927,"[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,15000000,"[{""id"": 18, ""name"": ""Drama""}]",http://www.dreamworks.com/ab/,"[{""id"": 255, ""name"": ""male nudity""}, {""id"": 29...",en,American Beauty,"Lester Burnham, a depressed suburban father in...",80.878605,"[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...",...,356296601,122.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Look closer.,American Beauty,7.9,3313,"[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."


In [ ]:
df.shape

(4803, 22)

In [ ]:
df.dtypes

,0
id,int64
budget,int64
genres,object
homepage,object
keywords,object
original_language,object
original_title,object
overview,object
popularity,float64
production_companies,object


In [ ]:
import pandas as pd
import ast

def clean_json_columns(df, column_name, key='name'):
    """
    Cleans a column with JSON-like lists of dictionaries into a comma-separated string of specified key values.

    Parameters:
        df (DataFrame): The DataFrame containing the column to clean.
        column_name (str): The name of the column to clean.
        key (str): The key to extract values from the dictionaries. Default is 'name'.
    """
    df[column_name] = df[column_name].fillna('[]')  # Handle missing values
    df[column_name] = df[column_name].apply(lambda x: ', '.join([item[key] for item in ast.literal_eval(x)]))
    return df

# Specify the columns to clean and the keys to extract
columns_to_clean = {
    'genres': 'name',
    'keywords': 'name',
    'production_companies': 'name',
    'production_countries': 'name',
    'spoken_languages': 'name'
}

# Apply the cleaning function to all specified columns
for column, key in columns_to_clean.items():
    df = clean_json_columns(df, column, key)

# Print to verify the changes
print(df[['genres', 'keywords', 'production_companies', 'production_countries', 'spoken_languages']].head())


                               genres  \
0                       Crime, Comedy   
1  Adventure, Action, Science Fiction   
2                   Animation, Family   
3              Comedy, Drama, Romance   
4                               Drama   

                                            keywords  \
0  hotel, new year's eve, witch, bet, hotel room,...   
1  android, galaxy, hermit, death star, lightsabe...   
2  father son relationship, harbor, underwater, f...   
3  vietnam veteran, hippie, mentally disabled, ru...   
4  male nudity, female nudity, adultery, midlife ...   

                                production_companies  \
0                        Miramax Films, A Band Apart   
1  Lucasfilm, Twentieth Century Fox Film Corporation   
2                            Pixar Animation Studios   
3                                 Paramount Pictures   
4                DreamWorks SKG, Jinks/Cohen Company   

       production_countries spoken_languages  
0  United States of America     

In [ ]:
df.head()

,id,budget,genres,homepage,keywords,original_language,original_title,overview,popularity,production_companies,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew
0,5,4000000,"Crime, Comedy",None,"hotel, new year's eve, witch, bet, hotel room,...",en,Four Rooms,It's Ted the Bellhop's first night on the job....,22.876230,"Miramax Films, A Band Apart",...,4300000,98.0,English,Released,Twelve outrageous guests. Four scandalous requ...,Four Rooms,6.5,530,"[{""cast_id"": 42, ""character"": ""Ted the Bellhop...","[{""credit_id"": ""52fe420dc3a36847f800012d"", ""de..."
1,11,11000000,"Adventure, Action, Science Fiction",http://www.starwars.com/films/star-wars-episod...,"android, galaxy, hermit, death star, lightsabe...",en,Star Wars,Princess Leia is captured and held hostage by ...,126.393695,"Lucasfilm, Twentieth Century Fox Film Corporation",...,775398007,121.0,English,Released,"A long time ago in a galaxy far, far away...",Star Wars,8.1,6624,"[{""cast_id"": 3, ""character"": ""Luke Skywalker"",...","[{""credit_id"": ""52fe420dc3a36847f8000437"", ""de..."
2,12,94000000,"Animation, Family",http://movies.disney.com/finding-nemo,"father son relationship, harbor, underwater, f...",en,Finding Nemo,"Nemo, an adventurous young clownfish, is unexp...",85.688789,Pixar Animation Studios,...,940335536,100.0,English,Released,"There are 3.7 trillion fish in the ocean, they...",Finding Nemo,7.6,6122,"[{""cast_id"": 8, ""character"": ""Marlin (voice)"",...","[{""credit_id"": ""52fe420ec3a36847f80006b1"", ""de..."
3,13,55000000,"Comedy, Drama, Romance",None,"vietnam veteran, hippie, mentally disabled, ru...",en,Forrest Gump,A man with a low IQ has accomplished great thi...,138.133331,Paramount Pictures,...,677945399,142.0,English,Released,"The world will never be the same, once you've ...",Forrest Gump,8.2,7927,"[{""cast_id"": 7, ""character"": ""Forrest Gump"", ""...","[{""credit_id"": ""52fe420ec3a36847f800076b"", ""de..."
4,14,15000000,Drama,http://www.dreamworks.com/ab/,"male nudity, female nudity, adultery, midlife ...",en,American Beauty,"Lester Burnham, a depressed suburban father in...",80.878605,"DreamWorks SKG, Jinks/Cohen Company",...,356296601,122.0,English,Released,Look closer.,American Beauty,7.9,3313,"[{""cast_id"": 6, ""character"": ""Lester Burnham"",...","[{""credit_id"": ""52fe420ec3a36847f8000809"", ""de..."


In [ ]:
df.dtypes

,0
id,int64
budget,int64
genres,object
homepage,object
keywords,object
original_language,object
original_title,object
overview,object
popularity,float64
production_companies,object


In [ ]:
df['genres'].head()

,genres
0,"Crime, Comedy"
1,"Adventure, Action, Science Fiction"
2,"Animation, Family"
3,"Comedy, Drama, Romance"
4,Drama


In [ ]:
def create_target_variable(df, target_genres=['Suspense', 'Thriller', 'Crime', 'Mystery', 'Horror', 'Action']):
    df['Recomended'] = df['genres'].apply(
        lambda x: 1 if any(genre.strip() in target_genres for genre in x.split(',')) else 0
    )
    return df

df = create_target_variable(df)

# Print the updated DataFrame
print(df[['genres','Recomended']])


                                  genres  Recomended
0                          Crime, Comedy           1
1     Adventure, Action, Science Fiction           1
2                      Animation, Family           0
3                 Comedy, Drama, Romance           0
4                                  Drama           0
...                                  ...         ...
4798                              Horror           1
4799               Comedy, Family, Drama           0
4800                     Thriller, Drama           1
4801                              Family           0
4802                         Documentary           0

[4803 rows x 2 columns]


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Text Preprocessing
X = df['overview'].fillna('')
y = df['Recomended']

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(stop_words='english', max_features=5000)),  # Text to TF-IDF
    ('classifier', LogisticRegression())  # Change to RandomForestClassifier() for RF
])

In [ ]:
pipeline.fit(X_train, y_train)

# Evaluate the model
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.78      0.82      0.80       509
           1       0.78      0.74      0.76       452

    accuracy                           0.78       961
   macro avg       0.78      0.78      0.78       961
weighted avg       0.78      0.78      0.78       961



In [ ]:
new_overviews = [
    "Kundan, a young man from Varanasi, is deeply in love with Zoya, a spirited woman who rejects him repeatedly. As she falls for another man with a hidden identity, Kundan's impulsive actions lead to tragic consequences, including betrayal, heartbreak, and death. Wracked with guilt, he seeks redemption by supporting Zoya's political ambitions, even as she conspires against him. A tale of love, revenge, and sacrifice, Kundan's journey explores the complexities of human emotions and the price of redemption."
]
predictions = pipeline.predict(new_overviews)
print(f"Predictions: {predictions}")

Predictions: [0]


In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def generate_movie_overview(movie_name, model_name="gpt2"):
    # Load pre-trained model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)

    # Prepare the prompt
    prompt = f"Write a 100-word overview for the movie titled '{movie_name}':\n"

    # Tokenize the prompt
    inputs = tokenizer.encode(prompt, return_tensors="pt")

    # Generate text
    outputs = model.generate(
        inputs,
        max_length=150,  # Length of the output
        num_beams=5,     # Beam search for better quality
        no_repeat_ngram_size=2,  # Avoid repetition
        early_stopping=True,
    )

    # Decode and return the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text[len(prompt):]  # Remove the prompt from the output


In [3]:
overview = generate_movie_overview('Money Hiest')
print("\nMovie Overview:\n")
print(overview)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.



Movie Overview:


http://www.netflix.com/browse/genre/100/Money-Hiest/money-hiest-1/ The movie begins with a scene in which a young man (played by the same actor as in the first movie) tells his mother that he is going to be a millionaire. He then goes on to say, "I want to live in a world where I can make money. I don't care if I'm rich or poor, I want my money to go to the people I love." The scene ends with the scene where the man tells the woman he wants to marry her. The woman responds by telling him that she wants


In [6]:
import wikipedia
from transformers import pipeline

# Step 1: Fetch movie plot from Wikipedia
def fetch_movie_plot(movie_name):
    try:
        # Search for the movie on Wikipedia
        page = wikipedia.page(movie_name)
        content = page.content

        # Extract the plot section
        if "Plot" in content:
            plot_index = content.find("Plot")
            # Attempt to extract until the next major section
            next_section_index = content.find("==", plot_index + 1)
            plot = content[plot_index:next_section_index].strip() if next_section_index != -1 else content[plot_index:].strip()
            return plot
        else:
            return "Plot section not found in the Wikipedia page."
    except Exception as e:
        return f"Error fetching Wikipedia page: {e}"

# Step 2: Summarize the plot
def summarize_plot(plot_text, model_name="t5-small"):
    try:
        summarizer = pipeline("summarization", model=model_name)
        # Summarize the plot
        summary = summarizer(plot_text, max_length=100, min_length=50, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return f"Error during summarization: {e}"

In [5]:
!pip install wikipedia

  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11679 sha256=c28603db4cdd4b7c92708c94b6cad82fbbe9b324fbf2280c4ed31dc0e8aecd01
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [10]:
def main():
    movie_name = input("Enter the movie name: ")
    print("\nFetching movie plot from Wikipedia...")

    # Step 1: Fetch plot
    plot = fetch_movie_plot(movie_name)
    if "Error" in plot or "not found" in plot:
        print(plot)
        return

    print("\nMovie Plot Extracted:")
    print(plot[:500], "...")  # Displaying the first 500 characters of the plot

    # # Step 2: Summarize the plot
    # print("\nSummarizing the plot...")
    # summary = summarize_plot(plot)
    # print("\nMovie Overview (100 Words):")
    # print(summary)


main()

Enter the movie name: Pushpa: The Rise

Fetching movie plot from Wikipedia...

Movie Plot Extracted:
Plot ...


In [28]:
import requests
from bs4 import BeautifulSoup
from transformers import pipeline

def get_movie_plot(movie_name):
    try:
        # Format the movie name for URL
        search_url = f"https://en.wikipedia.org/wiki/{movie_name.replace(' ', '_')}"
        response = requests.get(search_url)

        if response.status_code != 200:
            return f"Failed to fetch Wikipedia page for '{movie_name}'."

        # Parse the page content
        soup = BeautifulSoup(response.content, "html.parser")
        # print(soup.prettify())
        # Locate the plot section
        headers = soup.find_all('div', class_='mw-heading mw-heading2')
        print(headers)
        for header in headers:
          print(header.text)
          if header.text.strip() in ["Plot", "Plot[edit]"]:
            plot_section = header.find_next_sibling('p')
            plot_text = ""
            while plot_section and plot_section.name == 'p':
              plot_text += plot_section.text
              plot_section = plot_section.find_next_sibling()  # Continue to the next sibling

            return plot_text.strip()  # Return full aggregated plot text

        # return f"Plot section not found for '{movie_name}'."
    except Exception as e:
        return f"An error occurred: {e}"

def summarize_text(text, model_name="t5-small"):
    try:
        summarizer = pipeline("summarization", model=model_name)
        summary = summarizer(text, max_length=100, min_length=50, do_sample=False)
        return summary[0]['summary_text']
    except Exception as e:
        return f"Error during summarization: {e}"

movie_name = input("Enter the movie name: ")
print("\nFetching movie plot from Wikipedia...")

# Step 1: Fetch the plot
plot = get_movie_plot(movie_name)
if not plot or "not found" in plot.lower() or "error" in plot.lower():
    print("Error in Fetching plot")
    # return

print("\nMovie Plot Extracted:")
print(plot)  # Display the first 500 characters

# Step 2: Summarize the plot
print("\nSummarizing the plot...")
summary = summarize_text(plot)
print("\nMovie Overview (100 Words):")
print(summary)

Enter the movie name: Singham Again

Fetching movie plot from Wikipedia...
[<div class="mw-heading mw-heading2"><h2 id="Plot">Plot</h2></div>, <div class="mw-heading mw-heading2"><h2 id="Cast">Cast</h2></div>, <div class="mw-heading mw-heading2"><h2 id="Production">Production</h2></div>, <div class="mw-heading mw-heading2"><h2 id="Music">Music</h2></div>, <div class="mw-heading mw-heading2"><h2 id="Release">Release</h2></div>, <div class="mw-heading mw-heading2"><h2 id="Reception">Reception</h2></div>, <div class="mw-heading mw-heading2"><h2 id="Notes">Notes</h2></div>, <div class="mw-heading mw-heading2"><h2 id="References">References</h2></div>, <div class="mw-heading mw-heading2"><h2 id="External_links">External links</h2></div>]
Plot
Error in Fetching plot

Movie Plot Extracted:
DCP Bajirao Singham, is now married to Avni Kamat, who works in the Cultural Ministry, and has a son, Shaurya. He is transferred from Mumbai to Srinagar for three years. Once there, he is attacked by the Ch

Token indices sequence length is longer than the specified maximum sequence length for this model (1250 > 512). Running this sequence through the model will result in indexing errors



Movie Overview (100 Words):
DCP Bajirao Singham is married to avni Kamat, who works in the Cultural Ministry . he is attacked by the chief of Lashkar, Omar Hafeez, who fled from Pakistan to Sri Lanka . his sons, Riyaaz and Raza's plans were thwarted .
